In [14]:
#!pip install deepface
#!pip install tf-keras
#!pip install scikit-learn
#!pip install matplotlib

In [15]:
from deepface import DeepFace
from PIL import Image, ImageDraw
from IPython.display import display
import os
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

i=0

Define the folder and image name

In [16]:
folder_name = "../dataset/images/original/"
modified_folder_name = "../dataset/images/processed/"
cropped_folder_name = "../dataset/images/cropped/"

# Create the processed and cropped folders if they don't exist
os.makedirs(modified_folder_name, exist_ok=True)
os.makedirs(cropped_folder_name, exist_ok=True)

for image_name in os.listdir(folder_name):
    # Only process if the file is an image (e.g., PNG, JPG)
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Load image
        image_path = os.path.join(folder_name, image_name)
        image = Image.open(image_path)

        # Analyze the image
        analysis = DeepFace.analyze(img_path=image_path, detector_backend='retinaface', enforce_detection=False)
        
        # Draw bounding boxes for each face detected
        draw = ImageDraw.Draw(image)
        for idx, item in enumerate(analysis):
            region = item['region']
            x, y, w, h = region['x'], region['y'], region['w'], region['h']

            # Draw rectangle on the original image
            draw.rectangle([(x, y), (x + w, y + h)], outline='green', width=4)

            # Crop the detected face
            cropped_face = image.crop((x, y, x + w, y + h))
            cropped_face_path = os.path.join(cropped_folder_name, f"{image_name}_face_{idx + 1}.png")
            cropped_face.save(cropped_face_path)
            print(f"Cropped face saved as {cropped_face_path}")

        # Save the image with bounding boxes
        modified_image_path = os.path.join(modified_folder_name, "MOD_" + image_name)
        image.save(modified_image_path)
        
        # Optionally, display the processed image
        modified_image = Image.open(modified_image_path)
        #display(modified_image)


Action: race: 100%|██████████| 4/4 [00:00<00:00,  7.67it/s]  


Cropped face saved as ../dataset/images/cropped/mix_mask_3.jpg_face_1.png
Cropped face saved as ../dataset/images/cropped/mix_mask_3.jpg_face_2.png
Cropped face saved as ../dataset/images/cropped/mix_mask_3.jpg_face_3.png
Cropped face saved as ../dataset/images/cropped/mix_mask_3.jpg_face_4.png
Cropped face saved as ../dataset/images/cropped/mix_mask_3.jpg_face_5.png


Action: race: 100%|██████████| 4/4 [00:00<00:00,  6.07it/s]  


Cropped face saved as ../dataset/images/cropped/w_mask_1.png_face_1.png


Action: race: 100%|██████████| 4/4 [00:00<00:00,  7.46it/s]  


...

Cropped face saved as ../dataset/images/cropped/test3.png_face_1.png
Cropped face saved as ../dataset/images/cropped/test3.png_face_2.png


Action: race: 100%|██████████| 4/4 [00:00<00:00,  7.63it/s]  


Cropped face saved as ../dataset/images/cropped/test2.png_face_1.png
Cropped face saved as ../dataset/images/cropped/test2.png_face_2.png


Action: race: 100%|██████████| 4/4 [00:00<00:00,  6.89it/s]  

Cropped face saved as ../dataset/images/cropped/w_mask_2.png_face_1.png
Cropped face saved as ../dataset/images/cropped/w_mask_2.png_face_2.png


## NN creation and training

### Split the data into training and validation sets

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Paths to directories
dataset_dir = '../dataset'
train_dir = '../dataset_split/train'
val_dir = '../dataset_split/val'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

mask_dir = os.path.join(dataset_dir, 'mask')
no_mask_dir = os.path.join(dataset_dir, 'no_mask')

# Function to copy files to target directories
def copy_files(file_list, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    for file in file_list:
        shutil.copy(file, target_dir)

# Split the dataset
def split_and_copy():
    mask_train_files, mask_val_files = split_dataset(mask_dir)
    no_mask_train_files, no_mask_val_files = split_dataset(no_mask_dir)

    # Copy files to train/val directories
    copy_files(mask_train_files, os.path.join(train_dir, 'mask'))
    copy_files(mask_val_files, os.path.join(val_dir, 'mask'))
    copy_files(no_mask_train_files, os.path.join(train_dir, 'no_mask'))
    copy_files(no_mask_val_files, os.path.join(val_dir, 'no_mask'))

split_and_copy()

# Data augmentation and generators
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
epochs = 100
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

i+=1

# Save the model
model.save('mask_detector_model',i,'.h5')

# Print training summary
print("Model trained and saved as 'mask_detector_model.h5'")


Found 3274 images belonging to 2 classes.
Found 818 images belonging to 2 classes.
Epoch 1/100
103/103 [==============================] - 68s 653ms/step - loss: 0.4880 - accuracy: 0.7816 - val_loss: 0.3784 - val_accuracy: 0.8496
Epoch 2/100
103/103 [==============================] - 66s 643ms/step - loss: 0.3575 - accuracy: 0.8592 - val_loss: 0.3100 - val_accuracy: 0.8729
Epoch 3/100
103/103 [==============================] - 76s 736ms/step - loss: 0.3338 - accuracy: 0.8638 - val_loss: 0.2632 - val_accuracy: 0.9071
Epoch 4/100
103/103 [==============================] - 72s 690ms/step - loss: 0.2996 - accuracy: 0.8867 - val_loss: 0.2412 - val_accuracy: 0.9034
Epoch 5/100
103/103 [==============================] - 69s 665ms/step - loss: 0.3012 - accuracy: 0.8833 - val_loss: 0.2007 - val_accuracy: 0.9279
Epoch 6/100
103/103 [==============================] - 69s 664ms/step - loss: 0.2932 - accuracy: 0.8891 - val_loss: 0.2188 - val_accuracy: 0.9205
Epoch 7/100
103/103 [====================

TypeError: can only concatenate str (not "int") to str

In [19]:
model.save('mask_detector_model.h5')

In [20]:
# Paths to the training and validation datasets
train_dir = 'dataset/train'
val_dir = 'dataset/val'

# Image size (resize all images to the same size)
image_size = (128, 128)

# Rescale the pixel values for training set (without augmentation)
train_datagen = ImageDataGenerator(rescale=1./255)

# Rescale the pixel values for validation set (no augmentation)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load the training dataset from the directory
train_generator = train_datagen.flow_from_directory(
    train_dir,  # Directory for the training images
    target_size=image_size,  # Resize the images
    batch_size=32,
    class_mode='binary',  # Binary classification (mask/no mask)
    shuffle=True  # Shuffle training images
)

# Load the validation dataset from the directory
validation_generator = validation_datagen.flow_from_directory(
    val_dir,  # Directory for the validation images
    target_size=image_size,  # Resize the images
    batch_size=32,
    class_mode='binary',  # Binary classification (mask/no mask)
    shuffle=False  # Do not shuffle validation images
)

# Build the CNN model
model = models.Sequential([
    # Convolutional Layer 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    
    # Convolutional Layer 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Convolutional Layer 3
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Flatten the output to feed into fully connected layers
    layers.Flatten(),
    
    # Dense Layer 1
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    
    # Output Layer: Sigmoid activation for binary classification
    layers.Dense(1, activation='sigmoid')
])

# Compile the model with binary crossentropy loss (for binary classification) and Adam optimizer
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Number of epochs to train the model
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

i=1
# Save the trained model
model.save('face_mask_classifier_no_augmentatio_'+i+'.h5')
i+=1
# Plot accuracy and loss curves
plt.figure(figsize=(12, 4))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/train'

### Test the model on the test set

Load the model

In [21]:
## load the model
model = tf.keras.models.load_model('mask_detector_model.h5')

Test the model on the test set

In [22]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os

# Path to the test dataset
test_dir = '../dataset/images/cropped'

# Image size (same as used for training)
image_size = (128, 128)

# Threshold for binary classification
threshold = 0.5

# Function to preprocess a single image
def preprocess_image(image_path):
    # Load the image
    img = load_img(image_path, target_size=image_size)  # Resize the image
    img_array = img_to_array(img)  # Convert to NumPy array
    img_array = img_array / 255.0  # Rescale pixel values
    return np.expand_dims(img_array, axis=0)  # Add batch dimension

# Load the test dataset and make predictions
predictions = []
for image_name in os.listdir(test_dir):
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(test_dir, image_name)
        img_array = preprocess_image(image_path)  # Preprocess the image
        
        # Predict the class (probability) using the model
        pred_prob = model.predict(img_array)[0][0]  # Get the probability for the "mask" class
        pred_class = "Mask" if pred_prob >= threshold else "No Mask"  # Apply the threshold
        
        # Store the result
        predictions.append((image_name, pred_prob, pred_class))

# Display results
print("Predictions:")
for image_name, pred_prob, pred_class in predictions:
    print(f"Image: {image_name}, Probability: {pred_prob:.2f}, Predicted Class: {pred_class}")


ValueError: in user code:

    File "/home/andre/Documents/Image-and-Speech-Recognition-Project/.venv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 2436, in predict_function  *
        return step_function(self, iterator)
    File "/home/andre/Documents/Image-and-Speech-Recognition-Project/.venv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 2421, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/andre/Documents/Image-and-Speech-Recognition-Project/.venv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 2409, in run_step  **
        outputs = model.predict_step(data)
    File "/home/andre/Documents/Image-and-Speech-Recognition-Project/.venv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 2377, in predict_step
        return self(x, training=False)
    File "/home/andre/Documents/Image-and-Speech-Recognition-Project/.venv/lib/python3.12/site-package